In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.9 MB/s eta 0:00:00


In [3]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.5 MB/s eta 0:00:00


In [4]:
!pip install scikeras

In [5]:
# Import libraries
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/3_CTDC/Copy of ctdc_main_positive_features.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/3_CTDC/Copy of ctdc_main_negative_features.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/3_CTDC/Copy of ctdc_validation_positive_features.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/3_CTDC/Copy of ctdc_validation_negative_features.csv")

# Prepare features and labels for training
X_train = pd.concat([main_p, main_n])
y_train = pd.concat([pd.Series(1, index=main_p.index), pd.Series(0, index=main_n.index)])


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [6]:
# Define a cross-validation accuracy function
def cross_val_accuracy(model, X_train, y_train):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    return accuracies.mean()


In [7]:
#Support Vector Machine (SVM)
def objective_svm(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)

    model = SVC(kernel=kernel, C=C, gamma=gamma)
    return cross_val_accuracy(model, X_train, y_train)

study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=50)
print("SVM Best Parameters:", study_svm.best_params)
print("SVM Best Accuracy:", study_svm.best_value)


[I 2024-12-09 21:59:51,641] A new study created in memory with name: no-name-6a742559-9902-4cd2-b333-b3336f020af4
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
<ipython-input-7-cd4fae0f87b1>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)
[I 2024-12-09 21:59:52,668] Trial 0 finished with value: 0.8504957821518426 and parameters: {'kernel': 'linear', 'C': 60922.4286653576, 'gamma': 0.0920494120178813}. Best is trial 0 with value: 0.8504957821518426.
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in 

SVM Best Parameters: {'kernel': 'rbf', 'C': 446.3324019549276, 'gamma': 0.03493619342872914}
SVM Best Accuracy: 0.9304055054018054


In [8]:
#Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)
print("Random Forest Best Parameters:", study_rf.best_params)
print("Random Forest Best Accuracy:", study_rf.best_value)


[I 2024-12-09 22:00:45,046] A new study created in memory with name: no-name-55e085ac-0fe5-4f2c-9755-407451b7a8b9
[I 2024-12-09 22:00:47,504] Trial 0 finished with value: 0.884005475802871 and parameters: {'n_estimators': 191, 'max_depth': 27, 'min_samples_split': 13, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.884005475802871.
[I 2024-12-09 22:00:51,776] Trial 1 finished with value: 0.888286221696019 and parameters: {'n_estimators': 281, 'max_depth': 16, 'min_samples_split': 8, 'min_samples_leaf': 9}. Best is trial 1 with value: 0.888286221696019.
[I 2024-12-09 22:00:56,160] Trial 2 finished with value: 0.8865768832322036 and parameters: {'n_estimators': 290, 'max_depth': 11, 'min_samples_split': 20, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.888286221696019.
[I 2024-12-09 22:00:59,478] Trial 3 finished with value: 0.9080583099008436 and parameters: {'n_estimators': 154, 'max_depth': 26, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 3 with value: 0

Random Forest Best Parameters: {'n_estimators': 206, 'max_depth': 27, 'min_samples_split': 2, 'min_samples_leaf': 1}
Random Forest Best Accuracy: 0.9295508361698979


In [9]:
#Decision Tree
def objective_dt(trial):
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective_dt, n_trials=50)
print("Decision Tree Best Parameters:", study_dt.best_params)
print("Decision Tree Best Accuracy:", study_dt.best_value)


[I 2024-12-09 22:03:37,535] A new study created in memory with name: no-name-941971ec-07c6-4ed9-bab5-b4bcf9da16ca
[I 2024-12-09 22:03:37,744] Trial 0 finished with value: 0.8539514577475209 and parameters: {'max_depth': 9, 'min_samples_split': 9, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8539514577475209.
[I 2024-12-09 22:03:37,983] Trial 1 finished with value: 0.8419231907651324 and parameters: {'max_depth': 19, 'min_samples_split': 2, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.8539514577475209.
[I 2024-12-09 22:03:38,198] Trial 2 finished with value: 0.8496559123871542 and parameters: {'max_depth': 11, 'min_samples_split': 17, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.8539514577475209.
[I 2024-12-09 22:03:38,355] Trial 3 finished with value: 0.8307532928814563 and parameters: {'max_depth': 5, 'min_samples_split': 13, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8539514577475209.
[I 2024-12-09 22:03:38,505] Trial 4 finished with value: 0.832

Decision Tree Best Parameters: {'max_depth': 26, 'min_samples_split': 4, 'min_samples_leaf': 2}
Decision Tree Best Accuracy: 0.8737309456859552


In [10]:
#k-Nearest Neighbors (k-NN)
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm
    )
    return cross_val_accuracy(model, X_train, y_train)

study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=50)
print("k-NN Best Parameters:", study_knn.best_params)
print("k-NN Best Accuracy:", study_knn.best_value)


[I 2024-12-09 22:03:52,750] A new study created in memory with name: no-name-a928df62-6dcd-4604-940c-94e620e9f487
[I 2024-12-09 22:03:52,929] Trial 0 finished with value: 0.8908983276602044 and parameters: {'n_neighbors': 14, 'weights': 'distance', 'algorithm': 'auto'}. Best is trial 0 with value: 0.8908983276602044.
[I 2024-12-09 22:03:53,054] Trial 1 finished with value: 0.8994857185141335 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'auto'}. Best is trial 1 with value: 0.8994857185141335.
[I 2024-12-09 22:03:54,132] Trial 2 finished with value: 0.8771533224803907 and parameters: {'n_neighbors': 9, 'weights': 'uniform', 'algorithm': 'ball_tree'}. Best is trial 1 with value: 0.8994857185141335.
[I 2024-12-09 22:03:54,260] Trial 3 finished with value: 0.8994857185141335 and parameters: {'n_neighbors': 6, 'weights': 'distance', 'algorithm': 'auto'}. Best is trial 1 with value: 0.8994857185141335.
[I 2024-12-09 22:03:54,381] Trial 4 finished with value: 0.892615

k-NN Best Parameters: {'n_neighbors': 3, 'weights': 'distance', 'algorithm': 'brute'}
k-NN Best Accuracy: 0.9054980020719254


In [11]:
#Logistic Regression

def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    penalty = trial.suggest_categorical('penalty', ['l2'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])

    model = LogisticRegression(C=C, penalty=penalty, solver=solver)
    return cross_val_accuracy(model, X_train, y_train)

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=50)
print("Logistic Regression Best Parameters:", study_lr.best_params)
print("Logistic Regression Best Accuracy:", study_lr.best_value)


[I 2024-12-09 22:04:11,872] A new study created in memory with name: no-name-7d88b3df-970b-4ac2-a55d-88fe253006df
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2024-12-09 22:04:12,957] Trial 0 finished with value: 0.8788589610774012 and paramete

Logistic Regression Best Parameters: {'C': 21605.783291399806, 'penalty': 'l2', 'solver': 'saga'}
Logistic Regression Best Accuracy: 0.8882973212964334


In [12]:
#Naive Bayes
def objective_nb(trial):
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)

    model = GaussianNB(var_smoothing=var_smoothing)
    return cross_val_accuracy(model, X_train, y_train)

study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)
print("Naive Bayes Best Parameters:", study_nb.best_params)
print("Naive Bayes Best Accuracy:", study_nb.best_value)


[I 2024-12-09 22:07:06,259] A new study created in memory with name: no-name-a3080ed5-40b8-4912-a3df-6b056625686f
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-09 22:07:06,389] Trial 0 finished with value: 0.7628607370134676 and parameters: {'var_smoothing': 1.6494484637386902e-06}. Best is trial 0 with value: 0.7628607370134676.
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-09 22:07:06,488] Trial 1 finished with value: 0.7310714814266686 and

Naive Bayes Best Parameters: {'var_smoothing': 0.03675615975477465}
Naive Bayes Best Accuracy: 0.8359183069409502


In [13]:
#Gradient Boosting
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_gb = optuna.create_study(direction='maximize')
study_gb.optimize(objective_gb, n_trials=50)
print("Gradient Boosting Best Parameters:", study_gb.best_params)
print("Gradient Boosting Best Accuracy:", study_gb.best_value)


[I 2024-12-09 22:07:11,646] A new study created in memory with name: no-name-c3f1df8d-5b81-4199-84af-a373ea99c56b
<ipython-input-13-0f44ec352f28>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-13-0f44ec352f28>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-09 22:07:21,977] Trial 0 finished with value: 0.7063526713038331 and parameters: {'n_estimators': 53, 'learning_rate': 2.250960265807189e-05, 'max_depth': 11, 'subsample': 0.6717403598427534}. Best is trial 0 with value: 0.7063526713038331.
<ipython-input-13-0f44ec352f28>:4: Future

Gradient Boosting Best Parameters: {'n_estimators': 179, 'learning_rate': 0.09338031325465627, 'max_depth': 7, 'subsample': 0.7630157735116768}
Gradient Boosting Best Accuracy: 0.9218144146810715


In [14]:
#XGBoost

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("XGBoost Best Parameters:", study_xgb.best_params)
print("XGBoost Best Accuracy:", study_xgb.best_value)


[I 2024-12-09 22:24:52,960] A new study created in memory with name: no-name-a0596282-37f0-4448-afb5-10135e16f922
<ipython-input-14-ad894fc3f237>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-14-ad894fc3f237>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-09 22:24:57,349] Trial 0 finished with value: 0.9089240787331656 and parameters: {'n_estimators': 147, 'max_depth': 4, 'learning_rate': 0.049631403641793005, 'subsample': 0.70916603865474}. Best is trial 0 with value: 0.9089240787331656.
<ipython-input-14-ad894fc3f237>:6: FutureWar

XGBoost Best Parameters: {'n_estimators': 132, 'max_depth': 17, 'learning_rate': 0.09919949463119727, 'subsample': 0.8375782698875032}
XGBoost Best Accuracy: 0.929550836169898


In [15]:
# LightGBM
def objective_lgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 50)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        num_leaves=num_leaves
    )
    return cross_val_accuracy(model, X_train, y_train)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
print("LightGBM Best Parameters:", study_lgb.best_params)
print("LightGBM Best Accuracy:", study_lgb.best_value)


[I 2024-12-09 22:31:22,697] A new study created in memory with name: no-name-818bfc41-5b83-4d9c-8f02-2bb295dce065
<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3047
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 272
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[I 2024-12-09 22:31:23,330] Trial 0 finished with value: 0.8058383898179666 and parameters: {'n_estimators': 59, 'max_depth': 6, 'learning_rate': 4.648605481054575e-05, 'num_leaves': 37}. Best is trial 0 with value: 0.8058383898179666.
<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003126 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3047
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 272
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003741 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3062
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 267
[LightGBM] [Info] [binary:Bo

[I 2024-12-09 22:31:24,575] Trial 1 finished with value: 0.8247225099896405 and parameters: {'n_estimators': 179, 'max_depth': 10, 'learning_rate': 0.00010634738184584558, 'num_leaves': 21}. Best is trial 1 with value: 0.8247225099896405.
<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


...


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004651 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3047
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 272
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

[I 2024-12-09 22:32:33,542] Trial 49 finished with value: 0.8685585318928519 and parameters: {'n_estimators': 184, 'max_depth': 7, 'learning_rate': 0.006231888499073328, 'num_leaves': 38}. Best is trial 28 with value: 0.9286924670711855.


LightGBM Best Parameters: {'n_estimators': 117, 'max_depth': 16, 'learning_rate': 0.099236094816156, 'num_leaves': 41}
LightGBM Best Accuracy: 0.9286924670711855


In [16]:
#AdaBoost
def objective_ab(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    model = AdaBoostClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate
    )
    return cross_val_accuracy(model, X_train, y_train)

study_ab = optuna.create_study(direction='maximize')
study_ab.optimize(objective_ab, n_trials=50)
print("AdaBoost Best Parameters:", study_ab.best_params)
print("AdaBoost Best Accuracy:", study_ab.best_value)


[I 2024-12-09 22:32:33,562] A new study created in memory with name: no-name-ae728c87-1949-462c-bad9-da1de3a49754
<ipython-input-16-95971a9e6870>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: Futu

AdaBoost Best Parameters: {'n_estimators': 189, 'learning_rate': 0.09595264068693674}
AdaBoost Best Accuracy: 0.8496337131863253


In [17]:
#Neural Network

import optuna
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Load your dataset
# Replace this with your actual data loading process
X = np.random.rand(1000, 20)  # Dummy features
y = np.random.randint(0, 2, 1000)  # Dummy binary labels

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_nn(trial):
    # Define hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # Build the neural network
    model = Sequential()
    model.add(Dense(hidden_units, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dropout(dropout_rate))

    for _ in range(num_layers - 1):
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model (without validation data)
    history = model.fit(X, y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)

    # Evaluate the model
    train_accuracy = max(history.history['accuracy'])
    return train_accuracy

# Run Optuna study
study_nn = optuna.create_study(direction='maximize')
study_nn.optimize(objective_nn, n_trials=50)

# Print the best parameters and accuracy
print("Neural Network Best Parameters:", study_nn.best_params)
print("Neural Network Best Accuracy:", study_nn.best_value)

[I 2024-12-09 22:37:20,168] A new study created in memory with name: no-name-f0927c86-3fc4-428b-91a8-d08c9fb7d338
<ipython-input-17-565ccca4d2fe>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
<ipython-input-17-565ccca4d2fe>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activit

Neural Network Best Parameters: {'num_layers': 2, 'hidden_units': 384, 'dropout_rate': 0.1257538148335567, 'learning_rate': 0.0013643528191603891, 'batch_size': 32, 'epochs': 44}
Neural Network Best Accuracy: 1.0


In [18]:
#Multi-Layer Perceptron (MLP)

import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Load your dataset (replace this with your actual dataset)
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Normalize the dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_mlp(trial):
    # Define the hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 50, 200)

    # Define the MLP model
    model = MLPClassifier(
        hidden_layer_sizes=(hidden_units,) * num_layers,
        activation=activation,
        learning_rate_init=learning_rate,
        max_iter=epochs,
        batch_size=batch_size,
        solver='adam',
        random_state=42
    )

    # Evaluate using cross-validation
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Run Optuna study
study_mlp = optuna.create_study(direction='maximize')
study_mlp.optimize(objective_mlp, n_trials=50)

# Print results
print("MLP Best Parameters:", study_mlp.best_params)
print("MLP Best Accuracy:", study_mlp.best_value)

[I 2024-12-09 22:44:28,379] A new study created in memory with name: no-name-42fffb73-a81d-4749-ae46-41b62e4dc4bc
<ipython-input-18-5ee0db595401>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (116) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (116) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: S

MLP Best Parameters: {'num_layers': 3, 'hidden_units': 96, 'activation': 'logistic', 'learning_rate': 0.0013618007880811433, 'batch_size': 64, 'epochs': 86}
MLP Best Accuracy: 0.876


probabilites dataset

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Combine data and create labels
X_train = pd.concat([main_p, main_n], axis=0)
y_train = np.array([1] * len(main_p) + [0] * len(main_n))

X_validation = pd.concat([validation_p, validation_n], axis=0)
y_validation = np.array([1] * len(validation_p) + [0] * len(validation_n))

# Define models
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    #"CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42)
}

# Combined probabilities dataset
combined_probabilities = pd.DataFrame(index=X_validation.index)

# Train models and get probabilities
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    probabilities = model.predict_proba(X_validation)[:, 1]
    combined_probabilities[model_name] = probabilities

# Neural Network
def build_nn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train Neural Network
input_dim = X_train.shape[1]
nn_model = build_nn(input_dim)
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_validation, y_validation), verbose=1)

# Predict probabilities for Neural Network
nn_probabilities = nn_model.predict(X_validation).flatten()
combined_probabilities['NeuralNetwork'] = nn_probabilities

# Add ground truth
combined_probabilities['True_Label'] = y_validation

# Save the combined probabilities dataset
output_path = "CTDC_optuna.csv"
combined_probabilities.to_csv(output_path, index=False)
print(f"Combined probabilities dataset with Neural Network saved to {output_path}")


Training RandomForest...
Training GradientBoosting...
Training LogisticRegression...
Training DecisionTree...
Training NaiveBayes...
Training k-NN...
Training SVM...
Training XGBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [23:11:50] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003990 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3379
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 298
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training MLP...
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5794 - loss: 0.6621 - val_accuracy: 0.8500 - val_loss: 0.4463
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8534 - loss: 0.3946 - val_accuracy: 0.8600 - val_loss: 0.3218
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8931 - loss: 0.2618 - val_accuracy: 0.8800 - val_loss: 0.2815
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9313 - loss: 0.1959 - val_accuracy: 0.9033 - val_loss: 0.2615
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9365 - loss: 0.1683 - val_accuracy: 0.9267 - val_loss: 0.2487
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9533 - loss: 0.1242 - val_accuracy: 0.9367 - val_loss: 0.2409
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9737 - loss: 0.0839 - val_accuracy: 0.9267 - val_loss: 0.2358
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9693 - loss: 0.0803 - val_accuracy: 0.9267 - val_loss: 0.2448
Epo